In [1]:
from knowledgehub.api import KnowledgeHubAPI
import ipywidgets as w
from IPython.display import display, Javascript

import pandas as pd

from tqdm.notebook import tqdm

In [2]:
tqdm.pandas()

In [3]:
#api = KnowledgeHubAPI(server='TEST', client_secret='39c644b3-1f23-4d94-a71f-e0fb43ebd760')
api = KnowledgeHubAPI(server='DEV', client_secret='3db5a6d7-4694-48a4-8a2e-e9c30d78f9ab')

In [4]:
def getInchikey(smiles, pa_type):
    r = api.ChemistryService().paStandardize(smiles, pa_type)
    if r == None:
        return None
    else:
        return r[0]

# 1. Authentication
Use eTransafe's authentication service to get access to available services

In [5]:
username = w.Text(value='tester',placeholder='Knowledge Hub account', description='username:', disabled=False)
password = w.Password(value='', placeholder='Knowledge Hub password', description='password:', disabled=False)
loginBtn = w.Button(description='Login')
status = w.Output()

def on_button_clicked(_):
    if api.login(username.value, password.value) == False:
        print("Failed to login")
    else:
        print("successfully logged in")
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

loginBtn.on_click(on_button_clicked)
w.VBox([username, password, loginBtn])

successfully logged in


<IPython.core.display.Javascript object>

# 2. Get all preclinical compounds 

In [6]:
preclinical_cpds = api.eToxSys().getAllCompounds() + api.PreclinicalDb().getAllCompounds()

retrieving 0 of 1371 compounds
retrieving 1000 of 1371 compounds
retrieving 0 of 40 compounds


In [7]:
df_preclinical = pd.DataFrame(preclinical_cpds)

In [8]:
df_preclinical['name'] = df_preclinical['name'].str.lower()

In [9]:
df_preclinical[(~df_preclinical.name.isnull())&(df_preclinical.name.str.contains('-'))].shape

(32, 13)

In [10]:
df_preclinical[(~df_preclinical.name.isnull())].shape

(446, 13)

In [11]:
df_preclinical.head(1)

,id,additionalProperties,organisation,confidentiality,compoundIdentifier,name,inchi,inchiKey,smiles,studyIds,findingIds,createdDate,modifiedDate
0,1,[],Esteve,FULLY_SHAREABLE,E-5746DIMALEATE,None,InChI=1S/C22H32N6O/c1-2-29-19-18-28-21-9-4-3-8...,MSDGIWOGYAWOGX-UHFFFAOYSA-N,CCOCCn1c(nc2ccccc12)N1CCCN(CCCn2cccn2)CC1,"[1, 2]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2012-02-21T13:10:15.000Z,2016-10-19T14:28:56.000Z


In [12]:
df_preclinical['pa_inchikey'] =  df_preclinical.progress_apply(lambda x: getInchikey(x['smiles'], 'preclinical'), axis=1)

  0%|          | 0/1411 [00:00<?, ?it/s]

Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endpoint https://dev.toxhub.etransafe.eu/chemistryservice.kh.svc/v1/: 500
Cannot retrieve compoundIds from endp

In [13]:
df_preclinical.head(1)

,id,additionalProperties,organisation,confidentiality,compoundIdentifier,name,inchi,inchiKey,smiles,studyIds,findingIds,createdDate,modifiedDate,pa_inchikey
0,1,[],Esteve,FULLY_SHAREABLE,E-5746DIMALEATE,None,InChI=1S/C22H32N6O/c1-2-29-19-18-28-21-9-4-3-8...,MSDGIWOGYAWOGX-UHFFFAOYSA-N,CCOCCn1c(nc2ccccc12)N1CCCN(CCCn2cccn2)CC1,"[1, 2]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2012-02-21T13:10:15.000Z,2016-10-19T14:28:56.000Z,MSDGIWOGYAWOGX-UHFFFAOYSA-N


In [14]:
df_preclinical_inchi = df_preclinical[~df_preclinical.pa_inchikey.isnull()]

In [15]:
df_preclinical_inchi.pa_inchikey.value_counts()

BMPDWHIDQYTSHX-UHFFFAOYSA-N    3
JZCPYUJPEARBJL-UHFFFAOYSA-N    3
FWUQWDCOOWEXRY-ZDUSSCGKSA-N    3
BOFQWVMAQOTZIW-UHFFFAOYSA-N    2
NVWCZRPXYVDQEE-UHFFFAOYSA-N    2
                              ..
PXDNXJSDGQBLKS-UHFFFAOYSA-N    1
GEVVHUGVIJRPPF-UHFFFAOYSA-N    1
CAOBQTXBFNZHDA-XDHUDOTRSA-N    1
HYNBNUYQTQIHJK-UHFFFAOYSA-N    1
MMBJQNHQQHDAKU-UHFFFAOYSA-N    1
Name: pa_inchikey, Length: 1336, dtype: int64

In [16]:
df_preclinical_inchi[df_preclinical_inchi.pa_inchikey=='FWUQWDCOOWEXRY-ZDUSSCGKSA-N']

,id,additionalProperties,organisation,confidentiality,compoundIdentifier,name,inchi,inchiKey,smiles,studyIds,findingIds,createdDate,modifiedDate,pa_inchikey
389,390,[],AstraZeneca,FULLY_SHAREABLE,AZ_GGA_200011001,None,InChI=1S/C13H14N2/c14-13(11-6-2-1-3-7-11)10-12...,FWUQWDCOOWEXRY-ZDUSSCGKSA-N,N[C@@H](Cc1ccccn1)c1ccccc1,"[2705, 2706, 2707, 2708, 2709]","[581465, 581466, 581467, 581468, 581469, 58147...",2013-09-24T12:17:36.000Z,2016-10-19T14:29:15.000Z,FWUQWDCOOWEXRY-ZDUSSCGKSA-N
609,610,[],AstraZeneca,FULLY_SHAREABLE,AZ_GGA_229584209,None,InChI=1S/C13H14N2.2ClH/c14-13(11-6-2-1-3-7-11)...,KHJHFYAGQZYCLC-GXKRWWSZSA-N,Cl.Cl.N[C@@H](Cc1ccccn1)c1ccccc1,"[3945, 3946, 3947, 3948, 3949, 3950]","[816875, 816876, 816877, 816878, 816879, 81688...",2014-02-19T15:14:17.000Z,2016-10-19T14:29:23.000Z,FWUQWDCOOWEXRY-ZDUSSCGKSA-N
881,882,[],AstraZeneca,FULLY_SHAREABLE,AZ_GGA_202187213,None,InChI=1S/C13H14N2/c14-13(11-6-2-1-3-7-11)10-12...,FWUQWDCOOWEXRY-ZDUSSCGKSA-N,N[C@@H](Cc1ccccn1)c1ccccc1,"[4827, 4828, 4829]","[947947, 947948, 947949, 947950, 947951, 94795...",2014-09-10T10:13:25.000Z,2016-10-19T14:29:31.000Z,FWUQWDCOOWEXRY-ZDUSSCGKSA-N


# 3. Get all clinical compounds

In [18]:
clinical_cpds = api.Faers().getAllCompounds() + api.ClinicalTrials().getAllCompounds() + api.Medline().getAllCompounds() + api.DailyMed().getAllCompounds()

ValueError: invalid literal for int() with base 10: ''

In [33]:
df_clinical = pd.DataFrame(clinical_cpds)

In [34]:
df_clinical = df_clinical.drop_duplicates(['compoundIdentifier'])[['compoundIdentifier','name']]

In [35]:
df_clinical.compoundIdentifier.nunique()

5355

In [36]:
df_clinical['pa_inchikey'] =  df_clinical.progress_apply(lambda x: getInchikey(x['name'], 'clinical'), axis=1)

#### No inchikey returned by pa_standardize 

In [54]:
# ignore rows where a message was returned insead of an inchikey or when nothing was returned
# standard inchi keys are 27 long so it's handy to use this as a filter
df_clinical_no_inchi = df_clinical[(df_clinical.pa_inchikey.str.len()!=27)]

In [55]:
df_clinical_no_inchi.shape

(2506, 3)

In [39]:
df_clinical_no_inchi.pa_inchikey.value_counts()

U    122
A    107
P     69
M     69
S     58
E     39
O     25
G      5
C      3
Name: pa_inchikey, dtype: int64

In [56]:
df_clinical_inchi = df_clinical[~df_clinical.compoundIdentifier.isin(df_clinical_no_inchi.compoundIdentifier)]

In [57]:
df_clinical_inchi.shape

(2849, 3)

In [58]:
df_clinical_inchi.pa_inchikey.nunique()

2765

In [59]:
# some compounds get the same inchikey due to the rules applied by the chemistry service. That's okay!
df_clinical_inchi['pa_inchikey'].value_counts()

IKQCSJBQLWJEPU-UHFFFAOYSA-N    3
SNICXCGAKADSCV-JTQLQIEISA-N    3
ZFXYFBGIUFBOJW-UHFFFAOYSA-N    3
ULGZDMOVFRHVEP-RWJQBGPGSA-N    3
QYSXJUFSXHHAJI-YRZJJWOYSA-N    3
                              ..
CUHVIMMYOGQXCV-UHFFFAOYSA-N    1
VOKSWYLNZZRQPF-UHFFFAOYSA-N    1
WQZGKKKJIJFFOK-PHYPRBDBSA-N    1
AOJJSUZBOXZQNB-VTZDEGQISA-N    1
YLMGFJXSLBMXHK-UHFFFAOYSA-M    1
Name: pa_inchikey, Length: 2765, dtype: int64

# 4. Merge preclinical and clinical compounds

#### using preclinical inchikey returned by pa_standardize 

In [60]:
df_overlap = df_preclinical_inchi.merge(df_clinical_inchi, on = 'pa_inchikey', suffixes=['_preclinical', '_clinical'])
df_overlap.shape

(264, 16)

In [61]:
df_overlap.pa_inchikey.nunique()

235

#### using compoud names (preclinical compounds with no SMILEs can used here) 

In [62]:
df_overlap_name = df_preclinical.merge(df_clinical, on = 'name')
df_overlap_name.shape

(175, 16)

In [63]:
df_overlap_name[df_overlap_name.inchi.isnull()]

,id,additionalProperties,organisation,confidentiality,compoundIdentifier_x,name,inchi,inchiKey,smiles,studyIds,findingIds,createdDate,modifiedDate,pa_inchikey_x,compoundIdentifier_y,pa_inchikey_y
169,1361,[],Janssen Pharmaceutica N.V.,FULLY_SHAREABLE,JJ_PLS_0042,itraconazole,None,VHVPQPYKVGDNFY-ZPGVKDDISA-N,None,"[6483, 6484, 6485]","[1331698, 1331699, 1331700, 1331701, 1331702, ...",2016-10-28T09:59:58.000Z,2016-10-28T09:59:58.000Z,None,28031,VHVPQPYKVGDNFY-UHFFFAOYSA-N
170,1371,[],Janssen Pharmaceutica N.V.,FULLY_SHAREABLE,JJ_PLS_0106,phenobarbital,None,PZBPKYOVPCNPJY-UHFFFAOYSA-N,None,"[6500, 6501]","[1343990, 1343991, 1343992, 1343993, 1343994, ...",2016-10-28T12:01:10.000Z,2016-10-28T12:01:10.000Z,None,8134,DDBREPKUVSBGFI-UHFFFAOYSA-N
